In [ ]:
#Windows & Linux
# !pip install torch torchvision matplotlib opendatasets
#MacOS
# %pip install torch torchvision matplotlib opendatasets


import os
import torch
import torchvision.utils as utils
import torchvision.transforms as tt
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader,random_split
from torchvision.datasets import ImageFolder
import torch.nn.functional as F

import opendatasets as od

In [ ]:
dataset_url = 'https://www.kaggle.com/splcher/animefacedataset'
od.download(dataset_url)

In [ ]:
#Model Training
dir = "./animefacedataset"
image_size = 64
batch_size = 128
latent_size = 128
stats = (0.5,0.5,0.5),(0.5,0.5,0.5)

In [ ]:
#dataset Transforms
ds = ImageFolder(dir,transform=tt.Compose([
    tt.Resize(image_size),
    tt.CenterCrop(image_size),
    tt.ToTensor(),
    tt.Normalize(*stats)
]))

#training DataLoader
trdl = DataLoader(ds, batch_size, shuffle=True, num_workers =2, pin_memory=True) #You can change num_workers bassed on your system

In [ ]:
#to change Image into original form before
def denormalize(img):
  return img*stats[1][0] + stats[0][0]

def show_img(imgs,nmax):
  fig,ax = plt.subplots(figsize=(12,12))
  ax.set_xticks([]);ax.set_yticks([])
  plt.imshow(utils.make_grid(denormalize(imgs.detach()[:nmax]),nrow=8).permute(1,2,0))

def show_batch(dl,nmax=64):
  for batch,_ in dl:
    show_img(batch,nmax)
    break

In [ ]:
show_batch(trdl)

In [ ]:
def get_dev():
  if torch.cuda.is_available():
    return torch.device('cuda')
#   if torch.backends.mps.is_available():  #if you're using mac
#     return torch.device('mps')
  return torch.device('cpu')

def todev(data,dev):
  if isinstance(data, (list,tuple)):
    return [todev(i,dev) for i in data]
  return data.to(dev,non_blocking=True)

device = get_dev()
device

In [ ]:
#To move DataLoader from Cpu to Cuda
class MoveData():
  def __init__(self,dl,dev):
    self.dl = dl
    self.dev = dev

  def __iter__(self):
    for b in self.dl:
      yield todev(b,self.dev)

  def __len__(self):
    return len(self.dl)

trdl = MoveData(trdl,device)
    

In [ ]:
def cnb(input,output,kernel=4,stride=2,padding=1):
  return nn.Sequential(
      nn.Conv2d(input,output,kernel_size=kernel,stride = stride, padding=padding,bias=False),
      nn.BatchNorm2d(output),
      nn.LeakyReLU(0.2, inplace=True)
  )

def ctb(input,output,stride=2,padding=1):
  return nn.Sequential(
      nn.ConvTranspose2d(input,output,kernel_size=4,stride=stride, padding=padding,bias=False),
      nn.BatchNorm2d(output),
      nn.ReLU(True)
  )


#Discriminator model

dmodel = nn.Sequential(                                             #3 x 64 x 64
    cnb(3,64),                                                      #64 x 32 x 32
    cnb(64,128),                                                    #128 x 16 x 16
    cnb(128,256),                                                   #256 x 8 x 8
    cnb(256,512),                                                   #512 x 4 x 4

    nn.Conv2d(512,1,kernel_size=4,stride=1,padding=0,bias=False),   #1 x 1 x 1
    nn.Flatten(),
    nn.Sigmoid()
)


#Generator Model
                                
gmodel = nn.Sequential(                                                   #128 x 1 x 1
    ctb(latent_size,512,1,0),                                             #512 x 4 x 4
    ctb(512,256),                                                         #256 x 8 x 8
    ctb(256,128),                                                         #128 x 16 x 16
    ctb(128,64),                                                          #64 x 32 x 32

    nn.ConvTranspose2d(64,3,kernel_size=4,stride=2,padding=1,bias=False), #3 x 64 x 64
    nn.Tanh()
)


dmodel = todev(dmodel,device)
gmodel = todev(gmodel,device)



In [ ]:

#Discriminator Training Function
def trainD(images,opt):
  opt.zero_grad()
  rpreds = dmodel(images)
  ractual = torch.ones(images.size(0),1,device=device)
  rloss = F.binary_cross_entropy(rpreds,ractual)
  rscore = torch.mean(rpreds).item()

  finput = torch.randn(batch_size,latent_size,1,1,device=device)
  fimgs = gmodel(finput)

  factual = torch.zeros(finput.size(0),1,device=device)
  fpreds = dmodel(fimgs)
  floss = F.binary_cross_entropy(fpreds,factual)
  fscore = torch.mean(fpreds).item()

  loss = rloss + floss
  loss.backward()
  opt.step()
  return loss.item(),rscore,fscore

#Generator Training Function
def trainG(opt):
  opt.zero_grad()

  finput = torch.randn(batch_size,latent_size,1,1,device=device)

  imgs = gmodel(finput)
  preds = dmodel(imgs)
  actual = torch.ones(finput.size(0),1,device=device)
  loss = F.binary_cross_entropy(preds,actual)
  loss.backward()
  opt.step()
  return loss.item()

In [ ]:
outputdir="generated"
os.makedirs(outputdir,exist_ok=True)
modelsdir = "models"
os.makedirs(modelsdir,exist_ok=True)

#Function To save States After Each Epoch
def save(index,finput,show=False):
  fimgs= gmodel(finput)
  name="{0:0=4d}.png".format(index)
  utils.save_image(denormalize(fimgs),os.path.join(outputdir,name),nrow=8)
  torch.save(gmodel.state_dict(),os.path.join(modelsdir,"generator{0:0=4d}.bin".format(index)))
  torch.save(dmodel.state_dict(),os.path.join(modelsdir,"discriminator{0:0=4d}.bin".format(index)))
  print("{} Saved".format(name))
  if show:
    fig,ax = plt.subplots(figsize=(12,12))
    ax.set_xticks([]);ax.set_yticks([])
    ax.imshow(fimgs.cpu().detach(),nrow=8).permute(1,2,0)


In [ ]:
#output images are saved based on this input
fixed_input = torch.randn(64,latent_size,1,1,device=device)

In [ ]:
#saving first image with random model parameters
save(0,fixed_input)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def fit(epochs,lr=1e-5,start=1):
  torch.cuda.empty_cache()
  glosses = []
  dlosses = []
  rscores = []
  fscores = []

  dopt = torch.optim.Adam(dmodel.parameters(),lr=lr,betas=(0.5,0.999))
  gopt = torch.optim.Adam(gmodel.parameters(),lr=lr,betas=(0.5,0.999))

  for epoch in range(epochs):
    c=0             #just to print progress if you're using online notebook platforms you can remove this
    for imgs,_ in tqdm(trdl):
      dloss,rscore,fscore = trainD(imgs,dopt)

      gloss = trainG(gopt)
      c+=1          #just to print progress if you're using online notebook platforms you can remove this
      print("Epoch: {}\tbatches done: {}/{}\t percentage: {:.4f}% gloss: {}\tdloss: {}".format(epoch,c,len(trdl),100*c/len(trdl),gloss,dloss),end="\r") 

    glosses.append(gloss)
    dlosses.append(dloss)

    rscores.append(rscore)
    fscores.append(fscore)

    print("epoch: {}\tG-loss: {}\tD-loss: {}\tR-score: {}\tF-score: {}".format(epoch,gloss,dloss,rscore,fscore))

    save(epoch+start,fixed_input)

  return {"gloss": glosses,"dloss":dlosses,"rscore":rscores,"fscore":fscores}

In [ ]:
#Parameters for fit function
lr = 0.0002
epochs = 50

In [ ]:
#this is going to take hours 😵
his = fit(epochs,lr)

In [ ]:
#last Model
torch.save(gmodel.state_dict(),"generator.bin")
torch.save(dmodel.state_dict(),"discriminator.bin")

In [ ]:
plt.plot(his['dloss'],color='skyblue')
plt.plot(his['gloss'],color='green')
plt.legend(["D-Loss","G-Loss"])

In [ ]:
plt.plot(his['rscore'],color='skyblue')
plt.plot(his['fscore'],color='green')
plt.legend(["R-score","F-score"])

In [ ]:
'''
import os
import torch
import torch.nn as nn
import torchvision.utils as utils
import matplotlib.pyplot as plt

#parameters #DO NOT CHANGE unless you've made your own model with different parameters
batch_size = 128
latent_size = 128
stats = (0.5,0.5,0.5),(0.5,0.5,0.5)

#Device
def get_dev():
  if torch.cuda.is_available():
    return torch.device('cuda')
  if torch.backends.mps.is_available():  #if you're using mac
    return torch.device('mps')
  return torch.device('cpu')

def todev(data,dev):
  if isinstance(data, (list,tuple)):
    return [todev(i,dev) for i in data]
  return data.to(dev,non_blocking=True)

device = get_dev()
print(device)

#Model
def cnb(input,output,kernel=4,stride=2,padding=1):
  return nn.Sequential(
      nn.Conv2d(input,output,kernel_size=kernel,stride = stride, padding=padding,bias=False),
      nn.BatchNorm2d(output),
      nn.LeakyReLU(0.2, inplace=True)
  )
def ctb(input,output,stride=2,padding=1):
  return nn.Sequential(
      nn.ConvTranspose2d(input,output,kernel_size=4,stride=stride, padding=padding,bias=False),
      nn.BatchNorm2d(output),
      nn.ReLU(True)
  )
dmodel = nn.Sequential(cnb(3,64),cnb(64,128),cnb(128,256),cnb(256,512),nn.Conv2d(512,1,kernel_size=4,stride=1,padding=0,bias=False),nn.Flatten(),nn.Sigmoid())
gmodel = nn.Sequential(ctb(latent_size,512,1,0),ctb(512,256),ctb(256,128),ctb(128,64),nn.ConvTranspose2d(64,3,kernel_size=4,stride=2,padding=1,bias=False),nn.Tanh())
dmodel = todev(dmodel,device)
gmodel = todev(gmodel,device)
dmodel.load_state_dict(torch.load("./discriminator.bin"))
gmodel.load_state_dict(torch.load("./generator.bin"))

#Saving function
outputdir="generated"
os.makedirs(outputdir,exist_ok=True)
def denormalize(img):
  return img*stats[1][0] + stats[0][0]
def save_image(index,finput,show=False):
  fimgs= gmodel(finput)
  name="{0:0=4d}.png".format(index)
  utils.save_image(denormalize(fimgs),os.path.join(outputdir,name),nrow=8)
  print("{} Saved".format(name))
  if show:
    fig,ax = plt.subplots(figsize=(12,12))
    ax.set_xticks([]);ax.set_yticks([])
    ax.imshow(utils.make_grid(denormalize(fimgs).cpu().detach(),nrow=8).permute(1,2,0))
    
fimg = torch.randn(64, latent_size, 1, 1, device=device)
save_image(100,fimg, show=True)
'''